# 설정

In [1]:
# 파이썬 ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 사이킷런 ≥0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"

# 공통 모듈 임포트
import numpy as np
import os

# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(42)

# 깔끔한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "Evaluation Indicator"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    if tight_layout:
        plt.tight_layout()
    plt.save_fig(IMAGES_PATH, format=fig_extension, dpi=resolution)

# 평가

지금까지 분류 성능 평가에 정확도(정확하게 분류된 샘플이 비율)를 사용했고, 회귀 성능 평가에는 $R^2$을 사용했다. 그러나 주어진 데이터셋에 대한 지도 학습 모델의 성능을 재는 방법은 그 외에도 많다. 실전에서 애플리케이션에 따라 이런 평가 지표가 적합하지 않을 수 있으므로, 모델을 선택하고 매개변수를 튜닝할 때 올바른 지표를 선택하는 것이 중요하다.

## 최종 목표를 기억하라

평가 지표를 선택할 때 머신러닝 애플리케이션의 최종 목표를 기억해야 한다. 실제로 정확한 예측을 만드는 것뿐 아니라 큰 의사 결정 프로세스의 일부로 사용하는 데 더 중점을 둬야 할지 모른다. 머신러닝 평가 지표를 선택하기 전에 비지니스 지표라고 부르는 애플리케이션의 고차원적인 목표를 생각해야 한다. 어떤 머신러닝 애플리케이션에서 특정 알고리즘을 선택하여 나타난 결과를 <b>비지니스 임팩트</b><sup>business impact</sup>라고 한다. 고차원적인 목표는 교통사고를 피하거나 입원 환자 수를 줄이는 것일지 모른다. 또는 웹사이트에 더 많은 사용자를 유입시키거나 쇼핑몰에서 사용자의 소비를 늘리는 것일 수 있다. 모델을 선택하고 매개변수를 조정할 때, 이런 비지니스 지표에 긍정적인 영향을 주는 모델과 매개변수를 선택해야 한다. 많은 경우에 특정 모델이 비지니스에 미치는 영향은 실제 운영 시스템에 적용해야 알 수 있기 때문에 어려운 문제다.

개발 초기 단계에서 매개변수를 조정하기 위해 시험 삼아 모델을 운영 시스템에 곧바로 적용하기란 비지니스적으로나 개인적으로나 위험부담이 커서 현실적으로 불가능하다. 자율 주행 자동차의 보행자 회피 기능을 먼저 검증하지 않고 실전에 투입하여 평가한다고 생각해보자. 모델이 나쁘다면 보행자가 위험에 처하게 된다. 그래서 계산하기 쉬운 평가 지표를 이용한 대리 평가 방식을 종종 사용한다. 예를 들면 보행자와 보행자가 아닌 이미지를 분류하는 테스트를 수행해서 정확도를 측정할 수 있다. 이는 대체 방식이므로 평가가 가능하고 원래 비지니스 목적에 가장 가까운 지표를 찾아야 한다. 이 근사 지표는 모델을 평가할 때나 선택할 때마다 사용해야 한다. 평가의 결과는 하나의 숫자가 아닐 수 있지만, 선택한 모델의 예상 비지니스 임팩트를 나타내야 한다. 예를 들어 어떤 알고리즘을 적용하면 고객 수는 10% 늘지만 고객당 매출은 15% 줄어들 수 있다.

이번 절에서는 이진 분류의 특별한 사례에 대한 평가 지표를 먼저 이야기하고, 그다음으로 다중 분류를, 마지막으로 회귀에 대해 설명하겠다.

## 고려할 점

이진 분류는 실전에서 가장 널리 사용하고 개념도 쉬운 머신러닝 알고리즘이다. 하지만 이 간단한 작업을 평가하는 데에도 주의할 점이 많다. 여러 평가 지표를 들여다보기 전에, 정확도를 잘못 측정하는 경우에 대해 살펴보겠다. 이진 분류에는 양성 클래스와 음성 클래스가 있으며 양성 클래스가 우리의 관심 클래스다.

### 에러의 종류

잘못 분류한 샘플의 수가 원하는 정보의 전부는 아니므로, 정확도만으로 예측 성능을 측정하기에는 부족할 때가 종종 있다. 자동화 테스트로 암을 조기 발견하는 애플리케이션을 가정해보겠다. 테스트가 음성이면 건강하다는 뜻이다. 반대로 양성이면 추가 검사를 받아야 한다. 여기서 양성 테스트(암 진단)를 양성 클래스라고 하고 음성 테스트를 음성 클래스라고 한다. 모델이 항상 완벽하게 작동하는 것은 아니니, 잘못 분류할 때가 있다. 어떤 애플리케이션에서든 이런 분류 오류가 실제 현실에서 어떤 결과를 초래하는지 살펴봐야 한다.

예컨대 건강한 사람을 양성으로 분류하면 추가 검사를 받게 할 것이다. 이는 환자에게 비용 손실과 불편함을 가져다준다(그리고 약간의 스트레스도). 이와 같은 잘못된 양성 예측을 <b>거짓 양성</b><sup>false positive</sup>이라 한다. 반대로 암에 걸린 사람을 음성으로 분류하여 제대로 된 검사나 치료를 받지 못하게 할 때도 있다. 암을 발견하지 못하면 건강에 심각한 위협을 가하며 치명적일 수 있다. 이런 종류의 잘못된 음성 예측 오류를 <b>거짓 음성</b><sup>false negative</sup>이라 한다. 통계학에서는 거짓 양성을 타입Ⅰ에러, 거짓 음성을 타입Ⅱ에러라고도 한다. 암 진단 예에서는 거짓 음성을 최대한 피해야 하는 반면, 거짓 양성은 비교적 중요도가 낮다.

이 예는 특히 극단적이지만 일반적으로도 거짓 양성의 중요도와 거짓 음성의 중요도가 비슷한 경우는 매우 드물다. 상업적인 애플리케이션에서는 두 오류를 비용으로 환산하여, 예측 오류로 인한 금전적 손해를 측정한 값을 정확도 대신 사용하기도 한다. 이런 방식이 어떤 모델을 사용할지 비지니스 관점에서 판단하는 데 더 도움이 될 수 있다.

### 불균형 데이터셋

이 두 종류의 에러(거짓 양성과 거짓 음성)는 두 클래스 중 하나가 다른 것보다 훨씬 많을 때 더 중요하다. 실제로 이는 매우 흔한 상황이며, 좋은 예로는 어떤 아이템이 사용자에게 보여진 노출<sup>impression</sup> 데이터로 클릭을 예측하는 것이다. 아이템은 광고일 수도 있고, 관련 기사나 소셜 미디어 사이트에서 팔로우를 위해 추천하는 사람일 수도 있다. 목표는 특정 상품을 보여주면 사용자가 클릭을 할지(즉 관심 대상인지)를 예측하는 것이다. 인터넷에서 볼 수 있는 정보 대부분은 (특히 광고는) 클릭까지 이어지지 않는다. 그래서 사용자가 관심 있는 것을 클릭할 때까지 100개의 공고나 글을 보여줘야 할 수도 있다. 이때 클릭이 아닌 데이터 99개와 클릭 데이터 1개가 데이터셋으로 만들어진다. 다르게 말하면 샘플의 99%가 '클릭 아님'클래스에 속한다. 이렇게 한 클래스가 다른 것보다 훨씬 많은 데이터셋을 <b>불균형 데이터셋</b><sup>imbalanced datasets</sup> 또는 불균형 클래스의 데이터셋이라 한다. 현실에서는 불균형 데이터가 훨씬 많으며, 관심 대상인 이벤트의 빈도가 그렇지 않은 이벤트와 같거나 비슷한 경우는 드물다.

클릭을 99% 정확도로 예측하는 분류기를 만들었다고 해보자. 이는 무슨 뜻일까? 99% 정확도는 꽤 높아 보이지만 이는 불균형 클래스를 고려하지 못했다. 굳이 머신러닝 모델을 만들지 않고서도 무조건 '클릭 아님'으로 예측하면 그 정확도는 99%다. 하지만 불균형 데이터에서도 99% 정확도는 사실 매우 좋은 성능일 수 있다. 그래서 정확도로는 '무조건 클릭 아님' 모델과 '진짜 좋은 모델'을 구분하기 어렵다.

예를 위해서 digits 데이터셋을 사용해 숫자 9를 다른 숫자와 구분해서 9:1의 불균형한 데이터셋을 만들겠다.<sup><a id="a01" href="#p01">[1]</a></sup>

In [2]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

digits = load_digits()
y = digits.target == 9

X_train, X_test, y_train, y_test = train_test_split(
    digits.data, y, random_state=0)

항상 다수인 클래스(여기서는 '9 아님')를 예측값으로 내놓는 DummyClassifier를 사용해서 정확도를 계산해보겠다.<sup><a id="a02" href="#p02">[2]</a></sup>

In [4]:
from sklearn.dummy import DummyClassifier
dummy_majority = DummyClassifier(strategy='most_frequent').fit(X_train, y_train)
pred_most_frequent = dummy_majority.predict(X_test)
print("예측된 레이블의 레이블:", np.unique(pred_most_frequent))
print("테스트 점수: {:.2f}".format(dummy_majority.score(X_test, y_test)))

예측된 레이블의 레이블: [False]
테스트 점수: 0.90


거의 아무것도 학습하지 않고 90% 정확도를 얻었다. 누군가 자기 모델의 정확도가 90%라고 말하면 그 사람이 큰일을 해냈다고 생각할 수 있다. 하지만 문제에 따라서는 그저 무조건 한 클래스를 예측하기만 해도 될 수 있다. 실제 분류기를 사용한 것과 비교해보겠다.

In [5]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth=2).fit(X_train, y_train)
pred_tree = tree.predict(X_test)
print("테스트 점수: {:.2f}".format(tree.score(X_test, y_test)))

테스트 점수: 0.92


정확도로 보면 DecisionTreeClassifier가 더미 분류기보다 조금 나을 뿐이다. 이는 DecisionTreeClassifier를 잘못 사용했거나 이 문제에서는 정확도가 높은 측정 방법이 아님을 말해준다.

비교를 위해 LogisticRegression과 기본 DummyClassifier 분류기 두 개를 더 살펴보겠다. DummyClassifier는 무작위로 선택하므로 훈련 세트와 같은 비율의 예측값을 만든다.<sup><a id="a03" href="#p03">[3]</a></sup>

In [6]:
from sklearn.linear_model import LogisticRegression

dummy = DummyClassifier(strategy='stratified').fit(X_train, y_train)
pred_dummy = dummy.predict(X_test)
print("dummy 점수: {:.2f}".format(dummy.score(X_test, y_test)))

logreg = LogisticRegression(C=0.1, max_iter=1000).fit(X_train, y_train)
pred_logreg = logreg.predict(X_test)
print("logreg 점수: {:.2f}".format(logreg.score(X_test, y_test)))

dummy 점수: 0.82
logreg 점수: 0.98


무작위로 예측하는 더미 분류기는 (정확도로 봐서는) 확실히 결과가 안 좋다. 반면에 LogisticRegression은 매우 좋다. 하지만 더미 분류기조차도 80%를 맞췄다. 이런 결과가 실제로 유용한 것인지 판단하기가 매우 어렵다. 불균형 데이터셋에서 예측 성능을 정량화하는 데 정확도는 적절한 측정 방법이 아니기 때문이다. 이 장의 나머지에서는 모델 선택을 도와주는 다른 평가 지표를 살펴보겠다. 특히 pred_most_frequent와 pred_dummy처럼, 빈도나 무작위 기반 예측보다 얼마나 더 나은지 알려주는 평가 지표가 필요하다. 모델을 평가하는 지표라면 이런 비상식적인 예측은 피할 수 있어야 한다.

## 오차 행렬

이진 분류에서 성능 지표로 잘 활용되는 <b>오차행렬</b><sup>confusion matrix</sup>은 학습된 분류 모델이 예측을 수행하면서 얼마나 헷갈리고<sup>confused</sup> 있는지도 함께 보여주는 지표다. 즉, 이진 분류의 예측 오류가 얼마인지와 더불어 어떠한 유형의 예측 오류가 발생하고 있는지를 함께 나타내는 지표다.

오차 행렬은 다음과 같은 4분면 행렬에서 실제 레이블 클래스 값과 예측 레이블 클래스 값이 어떠한 유형을 가지고 매핑되는지를 나타낸다. 4분면의 왼쪽, 오른쪽을 예측된 클래스 값 기준으로 Negative와 Positive로 분류하고, 4분면의 위, 아래를 실제 클래스 값 기준으로 Negative와 Positive로 분류하면 예측 클래스와 실제 클래스의 값 유형에 따라 결전되는 TN, FP, FN, TP 형태로 오차 행렬의 4분면을 채울 수 있다. TN, FP, FN, TP 값을 다양하게 결합해 분류 모델 예측 성능의 오류가 어떠한 모습으로 발생하는지 알 수 있는 것이다.

<b>그림 1</b> 오차 행렬
<div style="text-align:center;">
    <img src="./images/Evaluation Indicator/오차 행렬.png">
</div>

TN, FP, FN, TP는 예측 클래스와 실제 클래스의 Positive 결정 값(값 1)과 Negative 결정 값(값 0)의 결합에 따라 결정된다. 예를 들어 TN은 True Negative의 의미이며 앞 True는 예측 클래스 값과 실제 클래스 값이 같다는 의미고 뒤의 Negative는 예측값이 Negative 값이라는 의미다. 즉, TN은 예측을 Negative 값 0으로 예측했는데, 실제 값도 Negative 값 0이라는 의미다. TN, FP, FN, TP 기호가 의미하는 것은 앞 문자 True/False는 예측값과 실제값이 '같은가/틀린가'를 의미한다. 뒤 문자 Negative/Positive는 예측 결과 값이 부정(0)/긍정(1)을 의미한다.

<ul>
    <li>TN은 예측값을 Negative 값 0으로 예측했고 실제 값 역시 Negative 값 0</li>
    <li>FP는 예측값을 Positive 값 1로 예측했는데 실제 값은 Negative 값 0</li>
    <li>FN은 예측값을 Negative 값 0으로 예측했는데 실제 값은 Positive 값 1</li>
    <li>TP는 예측값을 Positive 값 1로 예측했는데 실제 값 역시 Positive 값 1</li>
</ul>

# 미주

<b id="p01">1</b> y는 digits.target의 값이 0~8일 때는 False, 9일 때는 True로 만들어 이 예를 이진 분류의 문제로 바꾸었다. [↩](#a01)

<b id="p02">2</b> DummyClassifier는 실제 모델과 비교하기 위해 간단한 규칙을 지원하는 모델이다. strategy 옵션으로 지정할 수 있는 규칙은 클래스 레이블 비율에 맞게 예측하는 stratified, 가장 많은 레이블을 예측하는 most_frequent 등이 있다. 회귀에서 이에 상응하는 DummyRegressor가 있으며, 지원하는 규칙으로는 (기본값인) 평균값을 예측하는 mean과 중간값을 예측하는 median 등이 있다. [↩](#a02)

<b id="p03">3</b> DummyClassifier의 strategy로 지정한 stratified 방식은 클래스 레이블의 비율과 같은 비율로 예측 결과를 만들지만, 타깃 값 y_test와는 다르므로 정확도는 더 낮아진다. [↩](#a03)